In [30]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
product_df = pd.read_csv("converted_products.csv")
allowed_values_df = pd.read_csv("dietary_Allowances.csv")
# allowed_values_df.head()
product_df.dropna(axis=0)


,Brand,Size,Flavor,Rating,Price,CALORIES,TOTAL FAT,CHOLESTEROL,SODIUM,TOTAL CARBOHYDRATE,...,PHOSPHORUS,MAGNESIUM,VITAMIN C,VITAMIN D.2,NIACIN,VITAMIN B6,VITAMIN B12,NIACIN.1,BIOTIN,FOLATE
0,Optimum Nutrition,1 Lb.,Delicious Strawberry,9.3,$19.99,120,1500.0,40.0,50.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Optimum Nutrition,1 Lb.,Double Rich Chocolate,9.3,$19.99,120,1500.0,35.0,50.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Optimum Nutrition,1 Lb.,Vanilla Ice Cream,9.3,$19.99,120,1000.0,35.0,105.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Optimum Nutrition,2 Lbs.,Banana Cream,9.3,$29.99,120,1000.0,40.0,100.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Optimum Nutrition,2 Lbs.,Birthday Cake,9.3,$29.99,130,1500.0,35.0,150.0,5000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,Vital Proteins,9 Oz.,Strawberry Lemon,10.0,$25.00,70,0.0,0.0,100.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1364,Vital Proteins,9 Oz.,Tropical Hibiscus,10.0,$25.00,70,0.0,0.0,75.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1365,Vital Proteins,9 Oz.,Watermelon Mint,10.0,$25.00,70,0.0,0.0,100.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1366,Vital Proteins,14 Packets,Lavender Lemon,10.0,$34.00,50,0.0,0.0,55.0,3000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Input: Gender and Ingredient List
gender = "Male"  # Example gender input
ingredient_list = {"SODIUM": 150, "MAGNESIUM": 30, "FOLATE": 20}  # Example ingredient input


In [32]:
updated_ingredient_list = {}
for nutrient, value in ingredient_list.items():
    # Check if nutrient is present in allowed_values_df for the specified gender
    matching_rows = allowed_values_df[(allowed_values_df["Nutrient"] == nutrient) & (allowed_values_df["Gender"] == gender)]

    if not matching_rows.empty: 
        matching_row = matching_rows.iloc[0]
        allowed_range = matching_row["Allowance (mg)"]
        
        if not (value - 0.1 * value <= allowed_range <= value + 0.1 * value):
            print(f"Updating {nutrient} value to {allowed_range} (allowed range) for {gender}.")
            updated_ingredient_list[nutrient] = allowed_range
        else:
            updated_ingredient_list[nutrient] = value
    else:
        print(f"No information found for {nutrient} for {gender}. Keeping the original value.")
        updated_ingredient_list[nutrient] = value

print("Updated Ingredient List:")
print(updated_ingredient_list)

No information found for SODIUM for Male. Keeping the original value.
Updating MAGNESIUM value to 440.0 (allowed range) for Male.
Updating FOLATE value to 0.3 (allowed range) for Male.
Updated Ingredient List:
{'SODIUM': 150, 'MAGNESIUM': 440.0, 'FOLATE': 0.3}


In [33]:
ingredient_columns = product_df.columns[5:]
product_ingredients = product_df[ingredient_columns]

In [47]:
recommended_products = pd.DataFrame()
recommended_products_subset = pd.DataFrame()

updated_ingredient_df = pd.DataFrame([updated_ingredient_list])
common_ingredients = list(set(updated_ingredient_df.columns).intersection(set(product_ingredients.columns)))

# Ensure the order of columns matches between updated_ingredient_df and product_ingredients
updated_ingredient_df = updated_ingredient_df[common_ingredients]
product_ingredients = product_ingredients[common_ingredients]

updated_ingredient_df = updated_ingredient_df.dropna(axis=0)
product_ingredients = product_ingredients.dropna(axis=0)

if updated_ingredient_df.empty or product_ingredients.empty:
    print("Insufficient data after removing NaN values.")
else:
    cosine_similarities = cosine_similarity(updated_ingredient_df, product_ingredients)
    similarity_threshold = 0.8
    similar_product_indices = [i for i, sim in enumerate(cosine_similarities[0]) if sim > similarity_threshold]

    if similar_product_indices:
        recommended_products = product_df.iloc[similar_product_indices]
        recommended_products_subset = recommended_products[["Brand", "Rating", "Flavor", "Price"]]
        for _, row in recommended_products_subset.iterrows():
            print(f"Brand: {row['Brand']}, Rating: {row['Rating']}, Price: {row['Price']}")

    else:

        print("No similar products found.")

Brand: Primeval Labs, Rating: 10.0, Price: $42.99
Brand: Primeval Labs, Rating: 10.0, Price: $42.99
Brand: Gaspari Nutrition, Rating: 8.1, Price: $29.44
Brand: Gaspari Nutrition, Rating: 8.1, Price: $29.44
Brand: Gaspari Nutrition, Rating: 8.1, Price: $29.44
Brand: Gaspari Nutrition, Rating: 8.1, Price: $29.44
Brand: Gaspari Nutrition, Rating: 8.1, Price: $29.44
Brand: Gaspari Nutrition, Rating: 8.9, Price: $35.92
Brand: Gaspari Nutrition, Rating: 8.9, Price: $35.92
Brand: Gaspari Nutrition, Rating: 8.9, Price: $35.92
Brand: Gaspari Nutrition, Rating: 8.9, Price: $35.92
Brand: SNAC, Rating: 9.3, Price: $51.99
Brand: MHP, Rating: 8.4, Price: $43.99
Brand: Vital Proteins, Rating: nan, Price: $59.00
Brand: NLA for Her, Rating: 9.0, Price: $32.74


In [48]:
all_recommended_products = list(zip(recommended_products_subset['Brand'], recommended_products_subset['Flavor']))

# Calculate the novelty as the inverse of the frequency (popularity) of each recommended item
novelty_scores = {product: 1 / all_recommended_products.count(product) for product in set(all_recommended_products)}

# Calculate average novelty
average_novelty = sum(novelty_scores.values()) / len(novelty_scores) if len(novelty_scores) > 0 else 0

# Print the novelty
print(f"Average Novelty: {average_novelty:.2f}")

Average Novelty: 0.92


In [66]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Extract features for clustering
features_for_clustering = recommended_products.drop(["Brand", "Flavor", "Size", "Price", "Rating"], axis=1)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters,n_init=10, random_state=42)
predicted_clusters = kmeans.fit_predict(features_for_clustering)

silhouette_avg = silhouette_score(features_for_clustering, predicted_clusters)

# silhouette coefficient
print(f"Silhouette Coefficient: {silhouette_avg:.2f}")

print("Silhouette Coefficient = How well recommended products are similar to each")

Silhouette Coefficient: 0.87
Silhouette Coefficient = How well recommended products are similar to each
